In [52]:
from dotenv import load_dotenv
load_dotenv()
from openai import OpenAI
import os
from pinecone import Pinecone, ServerlessSpec

In [53]:
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
pc.create_index(
    name="rag", dimension=1536, metric="cosine", spec=ServerlessSpec(cloud="aws", region="us-east-1")
)

In [58]:
import json
data = json.load(open("reviews.json"))
data['reviews']

[{'professor': 'Dr. Emily Chen',
  'subject': 'Computer Science',
  'stars': 4,
  'reviews': 'Excellent teacher, explains complex concepts clearly. Assignments can be challenging.'},
 {'professor': 'Dr. Michael Johnson',
  'subject': 'Physics',
  'stars': 5,
  'reviews': 'Passionate about the subject, engaging lectures. Always available for office hours.'},
 {'professor': 'Prof. Sarah Williams',
  'subject': 'English Literature',
  'stars': 3,
  'reviews': 'Knowledgeable but sometimes disorganized. Grading can be inconsistent.'},
 {'professor': 'Dr. David Lee',
  'subject': 'Biology',
  'stars': 4,
  'reviews': 'Great lab instructor, brings real-world examples to class. Exams are fair.'},
 {'professor': 'Prof. Maria Garcia',
  'subject': 'History',
  'stars': 5,
  'reviews': 'Captivating storyteller, makes history come alive. Assignments encourage critical thinking.'},
 {'professor': 'Dr. Robert Taylor',
  'subject': 'Chemistry',
  'stars': 2,
  'reviews': 'Brilliant researcher but str

In [66]:
processed_data = []
client = OpenAI()

for review in data['reviews']:
    response = client.embeddings.create(
        input=review['reviews'],
        model="text-embedding-3-small",
    )
    embedding = response.data[0].embedding
    processed_data.append({
        "values": embedding,
        "id": review["professor"],
        "metadata": {
            "review": review["reviews"],
            "subject": review["subject"],
            "stars": review["stars"]
        }
    })

In [67]:
index = pc.Index('rag')
index.upsert(
    vectors=processed_data,
    namespace="ns1"
)

{'upserted_count': 20}

In [68]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'ns1': {'vector_count': 20}},
 'total_vector_count': 20}